# LangChain サンプル 1: Model I/O

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します

### BedrockLLM のサンプル

In [ ]:
from langchain_aws import BedrockLLM     

llm = BedrockLLM(model_id = "meta.llama3-8b-instruct-v1:0") # Bedrock では、mode_id でモデルを指定

result = llm.invoke (
    "子犬は犬です。子猫は",
    stop = "。"
)
print(result)

In [ ]:
from langchain_aws import BedrockLLM     

args = {"max_gen_len": 512, "temperature": 0.5, "top_p": 0.9}

llm = BedrockLLM(
    model_id = "meta.llama3-8b-instruct-v1:0",
    model_kwargs = args
) 

result = llm.invoke (
    "日本の首都は東京です。アメリカの首都は",
    stop = "。"
)
print(result)

### ChatBedrock のサンプル

In [ ]:
from langchain_aws import ChatBedrock      
from langchain_core.messages.human import HumanMessage 
from langchain_core.messages.system import SystemMessage 


chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" # Bedrock では、mode_id でモデルを指定
)

result = chat.invoke(
    [
        SystemMessage(content="あなたは優秀なアシスタントです。問いかけに回答してください。"),
        HumanMessage(content = "こんにちは！") # 人間からのメッセージを初期化してパラメータ指定
    ]
)
print(result.content)

################################
# HumanMessage を使わないパターン

messages = [
    (
        "system",
        "あなたは翻訳を行う有能なアシスタントです。ユーザーが入力した日本語を英語に翻訳して下さい。",
    ),
    ("human", "私は旅行に行くのが好きです。"),
]
ai_msg = chat.invoke(messages)
print(ai_msg)
print(ai_msg.content)


In [ ]:
from langchain_aws import ChatBedrock        
from langchain_core.messages.human import HumanMessage 
from langchain_core.messages.system import SystemMessage 

args = {"max_gen_len": 512, "temperature": 0.5, "top_p": 0.9}

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0", # Bedrock では、mode_id でモデルを指定
    model_kwargs = args
)

result = chat.invoke(
    [
        SystemMessage(content="あなたは優秀なアシスタントです。問いかけに回答してください。"),
        HumanMessage(content = "こんにちは！") # 人間からのメッセージを初期化してパラメータ指定
    ]
)
print(result.content)

### PromptTemplate のサンプル

In [ ]:
from langchain_aws import ChatBedrock        
from langchain_core.prompts.prompt import PromptTemplate 
from langchain_core.messages.human import HumanMessage 

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

prompt = PromptTemplate (
    template = "{country}の首都はどこですか？ 日本語で回答して下さい。",
    
    input_variables = [
        "country"
        ]
)

result = chat.invoke (
    [
        HumanMessage(content=prompt.format(country="日本")),
    ]
)

print(result.content)

In [ ]:
from langchain_aws import ChatBedrock        
from langchain_core.prompts.prompt import PromptTemplate  
from langchain_core.messages.human import HumanMessage 

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

prompt = PromptTemplate.from_template("{country}の首都はどこですか？ 日本語で回答して下さい。")

result = chat.invoke (
    [
        HumanMessage(content=prompt.format(country="アメリカ")),
    ]
)

print(result.content)

### Outputparser による CSV 形式での出力のサンプル

In [ ]:
from langchain_aws import ChatBedrock        
from langchain_core.messages.human import HumanMessage 
from langchain_core.output_parsers.list import CommaSeparatedListOutputParser 


output_parser = CommaSeparatedListOutputParser() # CSV 出力を指定する OutputParser の初期化

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0" 
)


result = chat.invoke (
    [
        HumanMessage(content="AWSが提供する有名なクラウドのサービスを３つあげて下さい"),
        HumanMessage(content=output_parser.get_format_instructions()), # CSV出力指示を追加
    ]
)
print(result.content)

output = output_parser.parse(result.content) # CSV形式を　List　に変換

for item in output:
    print("サービス：　" + item)

### Outputparser による Date 形式での出力のサンプル

In [ ]:
from unittest import result
from langchain.output_parsers import DatetimeOutputParser 
from langchain_aws import ChatBedrock        
from langchain_core.messages.human import HumanMessage 
from langchain_core.prompts.prompt import PromptTemplate  

#output_parser = DatetimeOutputParser() # デフォルトのフォーマットは、 '%Y-%m-%dT%H:%M:%S.%fZ'
output_parser = DatetimeOutputParser(format = '%Y-%m-%d') # Date形式出力を指定する OutputParser の初期化

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0"
)

prompt = PromptTemplate.from_template ("{person}の誕生日を教えて下さい。回答は日付だけを提示して下さい。")

result = chat.invoke(
    [
        HumanMessage(content=prompt.format(person="リンカーン大統領")),
        HumanMessage(content=output_parser.get_format_instructions())
    ]
)

print(result.content)

output = output_parser.parse(result.content)

print(output)
    



### Callback による Stream での出力のサンプル

In [ ]:
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 
from langchain_aws import ChatBedrock        
from langchain_core.messages.human import HumanMessage 
from langchain_core.prompts.prompt import PromptTemplate  

prompt = PromptTemplate (
    template = "おすすめの{place}の観光地を5つ、日本語で回答して下さい。",
    
    input_variables = [
        "place"
        ]
)

chat = ChatBedrock(
    model_id = "meta.llama3-8b-instruct-v1:0", 
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)


resp = chat.invoke ([
    HumanMessage(content = prompt.format(place="京都"))
])
